**Подключение библиотек и скриптов**

In [6]:
import numpy as np
import pandas as pd
import random

In [7]:
import warnings
warnings.filterwarnings('ignore')

**Пути к директориям и файлам**

In [8]:
TRAIN_DATASET_PATH = 'data/train.csv'
PREPARED_TRAIN_DATASET_PATH = 'data/train_prepared.csv'
TEST_DATASET_PATH = 'data/test.csv'

### Загрузка данных

**Описание датасета**

* **Id** - идентификационный номер квартиры
* **DistrictId** - идентификационный номер района
* **Rooms** - количество комнат
* **Square** - площадь
* **LifeSquare** - жилая площадь
* **KitchenSquare** - площадь кухни
* **Floor** - этаж
* **HouseFloor** - количество этажей в доме
* **HouseYear** - год постройки дома
* **Ecology_1, Ecology_2, Ecology_3** - экологические показатели местности
* **Social_1, Social_2, Social_3** - социальные показатели местности
* **Healthcare_1, Helthcare_2** - показатели местности, связанные с охраной здоровья
* **Shops_1, Shops_2** - показатели, связанные с наличием магазинов, торговых центров
* **Price** - цена квартиры

In [9]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
train_df.tail()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
9995,77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968,0.135650,B,B,46,7960,6,350.0,3,11,B,196684.316040
9996,6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000,0.000000,B,B,30,5562,0,NaN,0,5,A,189050.289571
9997,5123,27,1.0,47.939008,NaN,1.0,12,16.0,2015,0.072158,B,B,2,629,1,NaN,0,0,A,159143.805370
9998,5400,75,2.0,43.602562,33.840147,8.0,1,5.0,1961,0.307467,B,A,30,5048,9,325.0,2,5,B,181595.339808
9999,6306,128,1.0,38.666645,21.157874,8.0,7,17.0,1990,0.000000,B,B,27,4798,0,30.0,2,8,B,218714.077615


In [10]:
test_df = pd.read_csv(TEST_DATASET_PATH)
test_df.tail()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
4995,8180,11,3.0,67.133911,50.809797,6.0,5,9.0,1973,0.000170,B,B,36,5992,0,NaN,1,1,B
4996,4695,1,1.0,40.198472,21.807061,10.0,12,17.0,2017,0.007122,B,B,1,264,0,NaN,0,1,B
4997,5783,12,3.0,77.842178,48.282625,9.0,23,22.0,1989,0.090799,B,B,74,19083,2,NaN,5,15,B
4998,4780,62,2.0,81.305222,NaN,0.0,4,0.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A
4999,12504,30,2.0,60.555693,NaN,1.0,10,17.0,1977,0.000078,B,B,22,6398,141,1046.0,3,23,B


In [11]:
train_df.shape[1]-1 == test_df.shape[1]

True

### Приведение типов

In [12]:
train_df.dtypes

Id                 int64
DistrictId         int64
Rooms            float64
Square           float64
LifeSquare       float64
KitchenSquare    float64
Floor              int64
HouseFloor       float64
HouseYear          int64
Ecology_1        float64
Ecology_2         object
Ecology_3         object
Social_1           int64
Social_2           int64
Social_3           int64
Healthcare_1     float64
Helthcare_2        int64
Shops_1            int64
Shops_2           object
Price            float64
dtype: object

In [13]:
#Задание №1 
#привести Id и DistrictId к типу str

In [14]:
train_df['Id'].dtype

dtype('int64')

In [15]:
train_df['DistrictId'].dtype

dtype('int64')

In [16]:
train_df['Id'] = train_df['Id'].astype(str)

In [17]:
train_df['DistrictId'] = train_df['DistrictId'].astype(str)

In [18]:
train_df['Id'].dtype

dtype('O')

In [19]:
train_df['DistrictId'].dtype

dtype('O')

### Обзор данных

**Количественные переменные**

In [20]:
#Задание №2
#С помощью .describe получить первичное понимание того, что из себя представляют данные

In [21]:
train_df.describe()

,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


**Номинативные переменные**

In [22]:
#Задание №3
#С помощью .select_dtypes получить список категориальных признаков
#Также посчитать количество уникальных значений этих категориальных признаков

In [23]:
train_df_obj_features = train_df.select_dtypes(include='object')
train_df_obj_features.head()

,Id,DistrictId,Ecology_2,Ecology_3,Shops_2
0,14038,35,B,B,B
1,15053,41,B,B,B
2,4765,53,B,B,B
3,5809,58,B,B,B
4,10783,99,B,B,B


In [24]:
train_df['Ecology_2'].value_counts()

B    9903
A      97
Name: Ecology_2, dtype: int64

In [25]:
train_df['Ecology_2'].unique()

array(['B', 'A'], dtype=object)

In [26]:
train_df['Ecology_3'].value_counts()

B    9725
A     275
Name: Ecology_3, dtype: int64

In [27]:
train_df['Ecology_3'].unique()

array(['B', 'A'], dtype=object)

In [28]:
train_df['Shops_2'].value_counts()

B    9175
A     825
Name: Shops_2, dtype: int64

In [29]:
train_df['Shops_2'].unique()

array(['B', 'A'], dtype=object)

### Обработка выбросов

In [30]:
#Задание №4
#Обработать выбросы, подсказка: обратить внимание на признаки, связанные с параметрами домов, год постройки
#Выбросы в некоторых признаках можно найти с помощью квантилей.

In [31]:
train_df.describe()

,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [32]:
train_df['Rooms'].describe()

count    10000.000000
mean         1.890500
std          0.839512
min          0.000000
25%          1.000000
50%          2.000000
75%          2.000000
max         19.000000
Name: Rooms, dtype: float64

In [33]:
train_df['Rooms'].value_counts()

2.0     3880
1.0     3705
3.0     2235
4.0      150
5.0       18
0.0        8
10.0       2
19.0       1
6.0        1
Name: Rooms, dtype: int64

In [34]:
total_rooms_max_value = np.quantile(train_df['Rooms'], q=0.975)
total_rooms_max_value

3.0

In [35]:
total_rooms_min_value = np.quantile(train_df['Rooms'], q=0.025)
total_rooms_min_value

1.0

In [36]:
condition = (train_df['Rooms'] > total_rooms_max_value) | (train_df['Rooms'] < total_rooms_min_value)
train_df[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
77,2435,85,4.0,65.559834,47.696255,6.0,6,9.0,1976,0.037178,B,B,52,11217,1,2300.0,1,7,B,266989.016142
120,9703,15,4.0,98.686703,57.307058,15.0,10,17.0,2008,0.158249,B,B,21,5731,0,NaN,1,0,B,373510.057128
286,689,30,4.0,94.164934,60.322255,10.0,8,16.0,2005,0.000078,B,B,22,6398,141,1046.0,3,23,B,517817.162981
377,5927,57,10.0,59.056975,36.223072,10.0,22,22.0,2002,0.090799,B,B,74,19083,2,NaN,5,15,B,317265.323792
385,14888,69,4.0,98.385446,76.081776,9.0,14,22.0,1979,0.000991,B,B,33,6783,3,NaN,1,9,B,547029.902689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9607,2095,55,4.0,94.652198,58.206761,10.0,3,12.0,2000,0.041365,B,B,31,5569,0,NaN,1,4,B,351076.213145
9699,4971,169,5.0,117.855969,76.561318,9.0,12,14.0,2001,0.178917,B,B,18,3594,4,192.0,1,5,B,608674.130507
9852,16641,111,4.0,117.201937,69.898092,14.0,4,2.0,2017,0.093443,B,B,23,4635,5,3300.0,2,4,B,197593.112769
9910,16568,27,4.0,200.334539,201.627361,25.0,1,2.0,2013,0.041116,B,B,53,14892,4,NaN,1,4,B,528560.506016


In [37]:
train_df.loc[condition, 'Rooms'] = train_df['Rooms'].median()
train_df['Rooms'].describe()

count    10000.000000
mean         1.853000
std          0.756603
min          1.000000
25%          1.000000
50%          2.000000
75%          2.000000
max          3.000000
Name: Rooms, dtype: float64

In [38]:
train_df['Square'].describe()

count    10000.000000
mean        56.315775
std         21.058732
min          1.136859
25%         41.774881
50%         52.513310
75%         65.900625
max        641.065193
Name: Square, dtype: float64

In [39]:
total_rooms_max_value = np.quantile(train_df['Square'], q=0.975)
total_rooms_min_value = np.quantile(train_df['Square'], q=0.025)
condition = (train_df['Square'] > total_rooms_max_value) | (train_df['Square'] < total_rooms_min_value)
train_df.loc[condition, 'Square'] = train_df['Square'].median()
train_df['Square'].describe()

count    10000.000000
mean        55.108599
std         15.215053
min         32.105270
25%         42.432730
50%         52.513310
75%         64.591720
max        100.904263
Name: Square, dtype: float64

In [40]:
train_df['LifeSquare'].describe()

count    7887.000000
mean       37.199645
std        86.241209
min         0.370619
25%        22.769832
50%        32.781260
75%        45.128803
max      7480.592129
Name: LifeSquare, dtype: float64

In [41]:
total_rooms_max_value = np.quantile(train_df['LifeSquare'], q=0.975)
total_rooms_min_value = np.quantile(train_df['LifeSquare'], q=0.025)
condition = (train_df['LifeSquare'] > total_rooms_max_value) | (train_df['LifeSquare'] < total_rooms_min_value)
train_df.loc[condition, 'LifeSquare'] = train_df['LifeSquare'].median()
train_df['LifeSquare'].describe()

count    7887.000000
mean       37.199645
std        86.241209
min         0.370619
25%        22.769832
50%        32.781260
75%        45.128803
max      7480.592129
Name: LifeSquare, dtype: float64

In [42]:
train_df['KitchenSquare'].describe()

count    10000.000000
mean         6.273300
std         28.560917
min          0.000000
25%          1.000000
50%          6.000000
75%          9.000000
max       2014.000000
Name: KitchenSquare, dtype: float64

In [43]:
total_rooms_max_value = np.quantile(train_df['KitchenSquare'], q=0.975)
total_rooms_min_value = np.quantile(train_df['KitchenSquare'], q=0.025)
condition = (train_df['KitchenSquare'] > total_rooms_max_value) | (train_df['KitchenSquare'] < total_rooms_min_value)
train_df.loc[condition, 'KitchenSquare'] = train_df['KitchenSquare'].median()
train_df['KitchenSquare'].describe()

count    10000.000000
mean         5.536700
std          3.670668
min          0.000000
25%          1.000000
50%          6.000000
75%          8.000000
max         13.000000
Name: KitchenSquare, dtype: float64

In [44]:
train_df['Floor'].describe()

count    10000.000000
mean         8.526700
std          5.241148
min          1.000000
25%          4.000000
50%          7.000000
75%         12.000000
max         42.000000
Name: Floor, dtype: float64

In [45]:
total_rooms_max_value = np.quantile(train_df['Floor'], q=0.975)
total_rooms_min_value = np.quantile(train_df['Floor'], q=0.025)
condition = (train_df['Floor'] > total_rooms_max_value) | (train_df['Floor'] < total_rooms_min_value)
train_df.loc[condition, 'Floor'] = train_df['Floor'].median()
train_df['Floor'].describe()

count    10000.000000
mean         8.328400
std          4.610432
min          2.000000
25%          5.000000
50%          7.000000
75%         11.000000
max         21.000000
Name: Floor, dtype: float64

In [46]:
train_df['HouseFloor'].describe()

count    10000.000000
mean        12.609400
std          6.775974
min          0.000000
25%          9.000000
50%         13.000000
75%         17.000000
max        117.000000
Name: HouseFloor, dtype: float64

In [47]:
total_rooms_max_value = np.quantile(train_df['HouseFloor'], q=0.975)
total_rooms_min_value = np.quantile(train_df['HouseFloor'], q=0.025)
condition = (train_df['HouseFloor'] > total_rooms_max_value) | (train_df['HouseFloor'] < total_rooms_min_value)
train_df.loc[condition, 'HouseFloor'] = train_df['HouseFloor'].median()
train_df['HouseFloor'].describe()

count    10000.000000
mean        12.465700
std          6.376354
min          0.000000
25%          9.000000
50%         13.000000
75%         17.000000
max         25.000000
Name: HouseFloor, dtype: float64

### Обработка пропусков

In [48]:
#Задание №5
#Посчитать количество пропусков в каждом признаке
#Заменить пропуски медианой

In [49]:
train_df.isna().sum()

Id                  0
DistrictId          0
Rooms               0
Square              0
LifeSquare       2113
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     4798
Helthcare_2         0
Shops_1             0
Shops_2             0
Price               0
dtype: int64

In [50]:
train_df['LifeSquare_nan'] = 0
train_df

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_nan
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730,0
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063,0
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524,0
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217,0
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968,0.135650,B,B,46,7960,6,350.0,3,11,B,196684.316040,0
9996,6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000,0.000000,B,B,30,5562,0,NaN,0,5,A,189050.289571,0
9997,5123,27,1.0,47.939008,NaN,1.0,12,16.0,2015,0.072158,B,B,2,629,1,NaN,0,0,A,159143.805370,0
9998,5400,75,2.0,43.602562,33.840147,8.0,7,5.0,1961,0.307467,B,A,30,5048,9,325.0,2,5,B,181595.339808,0


In [51]:
train_df.loc[train_df['LifeSquare'].isna(), 'LifeSquare_nan'] = 1

In [52]:
train_df[train_df['LifeSquare_nan'] == 1]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_nan
7,11993,74,2.0,80.312926,NaN,0.0,14,0.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,221244.156664,1
8,5172,1,2.0,64.511437,NaN,1.0,9,17.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,229102.795999,1
21,11935,27,2.0,64.711835,NaN,1.0,15,1.0,1977,0.211401,B,B,9,1892,0,NaN,0,1,B,127200.026511,1
23,6641,54,3.0,52.513310,NaN,0.0,2,0.0,1977,0.006076,B,B,30,5285,0,645.0,6,6,B,571069.052600,1
25,5073,74,2.0,82.667915,NaN,1.0,16,17.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,244145.827069,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,16159,27,1.0,38.968338,NaN,1.0,10,1.0,1977,0.211401,B,B,9,1892,0,NaN,0,1,B,95529.569405,1
9986,2996,1,2.0,64.994245,NaN,1.0,9,22.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,200332.511658,1
9987,15623,23,2.0,55.620092,NaN,0.0,2,4.0,1977,0.034656,B,B,0,168,0,NaN,0,0,B,115797.553006,1
9994,4158,23,2.0,64.864198,NaN,1.0,17,17.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,171323.515653,1


In [53]:
median = train_df['LifeSquare'].median()
train_df['LifeSquare'] = train_df['LifeSquare'].fillna(median)

In [54]:
train_df['LifeSquare'].isna().sum()

0

In [55]:
median_Healthcare = train_df['Healthcare_1'].median()
train_df['Healthcare_1'].fillna(median_Healthcare, inplace=True)

In [56]:
train_df['Healthcare_1'].isna().sum()

0

In [57]:
train_df.drop(columns='LifeSquare_nan', inplace=True)

In [58]:
train_df.isna().sum()

Id               0
DistrictId       0
Rooms            0
Square           0
LifeSquare       0
KitchenSquare    0
Floor            0
HouseFloor       0
HouseYear        0
Ecology_1        0
Ecology_2        0
Ecology_3        0
Social_1         0
Social_2         0
Social_3         0
Healthcare_1     0
Helthcare_2      0
Shops_1          0
Shops_2          0
Price            0
dtype: int64

### Построение новых признаков

**OneHotEncoding**

In [59]:
#Задание №5
#Для категориальных переменных(несвязанных с Id) провести onehotencoding 

Построить новые 2 признака **DistrictSize, IsDistrictLarge**
где DistrictSize = количество уникальных значений DistrictId в train_df(например, DistrictId=35 встречалось в train_df 22 раза, значит DistrictSize=22)
Для этого создайте датафрейм district_size на основе информации из train_df. В датафрейме district_size должны быть 2 столбца: DistrictId, DistrictSize. После смерджить полученный датасет с train_df и сохранить результат в train_df.


IsDistrictLarge = это индикатор(1 или 0) того, что DistrictSize>100 или нет

Для примера возьму значения 1 строки:
Id = 14038; DistrictId = 35; DistrictSize = 22; IsDistrictLarge = 0

In [60]:
# pd.value_counts(train_df[['DistrictId']].values.ravel())
# train_df['DistrictId'].value_counts()
# numbr = pd.value_counts(train_df['DistrictId'].values.ravel())
numbr = train_df['DistrictId'].value_counts()
numbr.tail(110)


107    23
35     22
169    22
98     21
144    20
       ..
207     1
196     1
209     1
205     1
117     1
Name: DistrictId, Length: 110, dtype: int64

In [82]:
tmp2 = pd.DataFrame(columns=['DistrictId', 'DistrictSize'])
tmp2['DistrictId'] = train_df['DistrictId']
tmp2.loc[tmp2['DistrictSize'].isnull(), 'DistrictSize'] = train_df['DistrictId'].median()
tmp2

,DistrictId,DistrictSize
0,35,36
1,35,36
2,35,36
3,35,36
4,35,36
...,...,...
9995,136,36
9996,179,36
9997,202,36
9998,193,36


In [61]:
numbr['35']

22

In [88]:
district_size =  pd.DataFrame(columns=['DistrictId', 'DistrictSize'])
district_size['DistrictId'] = train_df['DistrictId']
# district_size['DistrictSize'] = (train_df['DistrictId'].map(train_df.groupby('DistrictId')['DistrictId'].agg(lambda x: x.value_counts().idxmax()))
district_size

,DistrictId,DistrictSize
0,35,NaN
1,35,NaN
2,35,NaN
3,35,NaN
4,35,NaN
...,...,...
9995,136,NaN
9996,179,NaN
9997,202,NaN
9998,193,NaN


In [63]:
train_df

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,900.0,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968,0.135650,B,B,46,7960,6,350.0,3,11,B,196684.316040
9996,6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000,0.000000,B,B,30,5562,0,900.0,0,5,A,189050.289571
9997,5123,27,1.0,47.939008,32.781260,1.0,12,16.0,2015,0.072158,B,B,2,629,1,900.0,0,0,A,159143.805370
9998,5400,75,2.0,43.602562,33.840147,8.0,7,5.0,1961,0.307467,B,A,30,5048,9,325.0,2,5,B,181595.339808


Построение признака **MedPriceByDistrict**

Создайте датафрейм med_price_by_district, который содержит информацию о медианной цене домов в одном районе и с одинаковым количеством комнат. После смерджите med_price_by_district и train_df по 2-м признакам

In [65]:
med_price_by_district = train_df.groupby(['DistrictId', 'Rooms'], as_index=False).median()[['DistrictId', 'Rooms', 'Price']]
med_price_by_district = pd.DataFrame(med_price_by_district)
med_price_by_district.rename(columns={'Price': 'Median_price'}, inplace=True)

med_price_by_district

,DistrictId,Rooms,Median_price
0,0,1.0,142799.434052
1,0,2.0,176210.812334
2,0,3.0,300610.359425
3,1,1.0,146735.671740
4,1,2.0,202706.090239
...,...,...,...
561,98,2.0,197798.451315
562,98,3.0,245090.013271
563,99,1.0,158539.052715
564,99,2.0,187726.339855


In [66]:
train_df = train_df.merge(med_price_by_district, on=['DistrictId', 'Rooms'])
train_df

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,Median_price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,900.0,0,11,B,184966.930730,212187.943646
1,8401,35,2.0,43.325817,22.487877,6.0,7,14.0,1969,0.089040,B,B,33,7976,5,900.0,0,11,B,194175.395111,212187.943646
2,14308,35,2.0,48.249779,29.473633,6.0,3,9.0,1971,0.089040,B,B,33,7976,5,900.0,0,11,B,181020.120527,212187.943646
3,15404,35,2.0,52.513310,66.600047,10.0,17,22.0,1998,0.089040,B,B,33,7976,5,900.0,0,11,B,559215.446005,212187.943646
4,14777,35,2.0,49.986281,29.976601,6.0,9,9.0,1970,0.089040,B,B,33,7976,5,900.0,0,11,B,212745.174808,212187.943646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,8953,136,1.0,40.901231,19.736669,9.0,16,14.0,1974,0.019509,B,B,37,7687,11,176.0,5,5,B,231960.056451,231960.056451
9996,3624,179,3.0,86.643824,57.727493,10.0,6,12.0,2003,0.037316,B,B,32,6748,93,562.0,4,10,B,452985.516682,452985.516682
9997,8396,202,2.0,42.353485,30.661180,6.0,4,6.0,1933,0.000000,B,B,30,7538,87,4702.0,5,5,B,502085.860162,502085.860162
9998,4050,193,2.0,46.034264,33.918107,5.0,4,5.0,1966,0.319809,B,B,25,4756,16,2857.0,5,8,B,181025.397125,181025.397125
